In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt
import itertools
 

I'm interested in using the tags that are present in the questions and lectures dataframe to help build a Cognitive Model of the users in this dataset: how do they learn the concept involved (what *are* the concepts invovled), what a particular user's learning state is at any given time, and how does the learning state affect their chances of answering a question right.

The tags are important because they **might** be a good representation of the individual skills involved in doing well on the TOEIC (the standardized test for which the users are practicing through this app).

In this notebook, I'm playing around with the concept of [Learning Factors Analysis](https://www.learnlab.org/uploads/mypslc/publications/learning_factor_analysis_5.2.pdf) to explore **whether some or all of the tags are a representation of learnable skills**.

# Learning Factors Analysis

The core idea behind Learning Factors Analysis is that a student's performance on a particular skill or concept improves as a power function of the amount of practice the student has had with that skill: 

$$D(o)=\alpha o^\gamma$$

Where:
 
- $o$ is the number of opportunities the student has had to practice the skill so far
- $D$ is some measure of difficulty the student can have with the task at hand, for example:
    - The odds (not probability!) of making a mistake
    - the amount of time it takes to complete the task or solve the problem
- $\alpha$ and $\gamma$ are factors which depend both on the student and the skill being learned:
    - $\alpha$ can be thought of as the starting-point difficulty, before the student has had any practice
    - $\gamma$ defines the rate at which the student's performance improves (and is hopefully negative, so that the performance actually improves)


For example, if $\alpha = 0.6$ and $\gamma = -0.2$:

In [ ]:
o = np.arange(1,100)
d = 0.6*o**(-0.2)
plt.plot(o,d)
plt.show()

In this example, the student started out with an $\alpha = 0.6$ chance of making a mistake, and after 100 practice opportinities, reduced that chance to about $0.25$.

Conveniently,this power function can be translated into linear form simply by taking the log of both sides:

$$\log(D) = \log(\alpha) + \gamma\log(o)$$

And linear forms means really easy regression!

- for real-valued measures of difficulty like time to completion, we could do linear regression
- for odds-based measures like the odds of making a mistake, logistic regression applies! As the Wikipedia article says, for logistic regresson,
> We assume a linear relationship between the predictor variables and the log-odds (also called logit) of the event that {\displaystyle Y=1}Y=1

In our case, $\log(D)$ is the log-odds of the event (of making a mistake), and our predictor variable is $\log(o)$

So the idea of Learning Factor Analysis is:
1. Use logistic regression to fit a learning curve onto the data of how many errors students make as they are solving problems
2. Based on how good or bad the fit is, try to improve our idea of what "skills" are, e.g. by splitting a single skill into two separate one.

The paper I linked above gives a specific example of a skill that works better when split into two sub-skills:

The data came from beginners learning how to program. One expert-defined skill was "making a function with one or more parameters". The Learning Factors Analysis found that this skill should be split into "make a function with one parameter" and "making a function with more than one parameters". It seems that conceptually, the skill of just passing data into a function is distinct from the skill of reasoning about how to manage *several* pieces of data.

# Are tags (learnable) skills?

In this notebook, I'm not going to actually apply the full LFA methodology for refining "skill tags" or Knowledge Components. I'm just going to calculate some learning curves, fit some power functions to them, and then eyeball whether some of the particular tags could benefit from being split into two.

For this notebook, I'm going to make the simplifying assumption that the difficulty of each attempt at practicing a skill depends *only* on that particular skill. This is only true if each attempt (exercise/question/encounter with concept) only tests that particular skill, and no other skills.

Luckily, as I discussed [in a previous notebook](https://www.kaggle.com/yanamal/questions-tags-and-lectures-riiid-metadata-eda), the Riiid dataset does have lots of skill tags that are only ever tested by themselves, not in conjunction with other tags! (It also has lots of tags that *are* always tested in conjunction with other tags, but that's a problem for future me. The LFA model does allow for "conjunctive" exercises which rely on several skills, but for now it's much simpler to ignore that possibility.)

In that same notebook, I made a tags.csv file with a row for each tag, and useful per-tag data like which questions use this tag, and how many of those questions test that particular skill by itself (i.e. don't have any other tags).

In [ ]:
# function to split a column with a space-separated list of integers into an actual python set. 
def split_column(df, col):
    return df[col].apply(lambda col_value: set([int(x) for x in str(col_value).split() if x != 'nan']))
    

In [ ]:
tags_df = pd.read_csv('../input/riiid-questions-tags-lectures-expanded-metadata/tags.csv')
tags_df['parts'] = split_column(tags_df, 'parts')
tags_df['questions'] = split_column(tags_df, 'questions')
tags_df.head()

Since this "fraction of multitagged questions" variable is almost always either really close to 1 or really close to 0, we can just look at the tags where it's close enough to 0. These tags are *nearly* always tested by themselves, not in conjunction with other skills.

In [ ]:
singleton_tags = list(tags_df[tags_df['fraction_multitagged'] < 0.5]['tag'])
print(f'{len(singleton_tags)} skill tags are (almost) always tested by themselves')

In another previous notebook, I split out train.csv from the Riiid dataset by tag: one .csv file for all the rows that are about questions/lectures with that particular tag (that notebook is not public because it's not very interesting).

Now I can load the .csv for each "singleton" tags into a separate dataframe, and post-process it a bit by adding a counter of how many "encounters" (or "opportunities") the student has had with the concept prior to answering each question.

In [ ]:
def load_tag_data(t):
    # load from expected filename for tag
    tag_learning_df = pd.read_csv(f'/kaggle/input/riiid-training-data-broken-out-per-tag/tag_{t}.csv', low_memory=False)
    # add column counting how many times each user has encountered this tag(including this current time)
    tag_learning_df['full_encounters'] = tag_learning_df.groupby('user_id').cumcount()+1
    tag_learning_df['feedback_encounters'] = tag_learning_df[
            (tag_learning_df['this_question_had_explanation']==True) |
            (tag_learning_df['this_question_had_explanation'].isnull())
        ].groupby('user_id').cumcount()+1
    return tag_learning_df

In [ ]:
%%time

per_tag_datasets = {}
for t in singleton_tags:
    print(t)
    per_tag_datasets[t] = load_tag_data(t)


Actually, as you can see in the code, the source data has an extra column, `this_question_had_explanation`. I calculate that in [this notebook](https://www.kaggle.com/yanamal/explanation-and-elapsed-time-for-current-question), and use it here to make a separate counter which doesn't count questions without feedback as "encounters".

I also always count lectures as encounters (I know a row is about a lecture when it has a null value for `this_question_had_explanation`). The idea is that anything a student might learn from is an 'encounter' which should help them progress down the learning curve. According to the data description, questions without feedback or explanation are usually part of a diagnostic mini-test that the user takes when they first start using the app. Since they never know exactly what they got right or wrong, they probably don't learn much from these diagnostic questions.

## Learning Curves
Now that we have the data loaded and the encounter counters calculated, we can plot the learning curves for some tags.

I'm using the **odds of making an error** as my difficulty/proficiency metric. Although we do also have access to the time it took to answer the question, it's not nearly as good a measurement of proficiency in this case. The student isn't required to answer the question correctly before moving on, so a student just guessing might take a short time, but still be very bad at the underlying skill. And even if they were required to get the right answer before moving on, in a multiple choice question, it's easy to just quickly try all the answers. The "time for task completion" measure is more for tasks where the student actually has to complete it to a satisfactory degree before the timer stops counting.

In [ ]:
def get_fraction_error(df, encounter_col, smoothing = 1):
    just_questions = df[df['answered_correctly']!=-1]
    correct = just_questions.groupby(encounter_col)['answered_correctly'].sum()
    total = just_questions.groupby(encounter_col)['answered_correctly'].count()
    if smoothing > 1:
        correct = correct.rolling(smoothing).sum()
        total = total.rolling(smoothing).sum()
    return (total-correct)/correct
    
def plot_encounters(df, encounter_col='feedback_encounters', ax=plt, limit=None, smoothing = 1, label=None, loglog=False):
    fraction_error = get_fraction_error(df, encounter_col, smoothing)
    
    x = list(fraction_error.index)
    y = list(fraction_error)
    if limit:
        x = x[:limit]
        y = y[:limit]
    
    if loglog:
        ax.loglog(x, y, label=label)
    else:
        ax.plot(x, y, label=label)

In [ ]:
plt.ylabel('error rate (odds ratio)')
plt.xlabel('number of encounters')
plot_encounters(per_tag_datasets[27])

Hmm. So the good news is that for this particular tag, the average error rate *does* seem to be going down with number of encounters.

But it's quite noisy, especially toward the end. It's also remarkable that some students seem to have answered questions or watched lectures about this concept up to 1600 times! And those students don't seem to be doing a whole lot better than others

We can add a bit of smoothing (by using a rolling window of encounters when calculating the odds ratio):

In [ ]:
plot_encounters(per_tag_datasets[27], smoothing=5)


It helps make the graph more readable up to about 800 encounters. But after 800 encounters it gets pretty noisy once again, and students seem to actually be doing worse than before.

If we zoom in on the graph up to just 500 encounters, it does look like a very plausible power curve - even without the smoothing:

In [ ]:
plot_encounters(per_tag_datasets[27], limit=500)


... But that's because I purposefully picked one of the best-fitting tags to show you first. Here is a comparison of that tag (27) to a couple of other interesting tags, all limited to the first 100 encounters:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 3))
plot_encounters(per_tag_datasets[27], label=27, ax=axes[0], limit=100)
plot_encounters(per_tag_datasets[23], label=23, ax=axes[1], limit=100)
plot_encounters(per_tag_datasets[152], label=152, ax=axes[2], limit=100)
[ax.legend() for ax in axes]
plt.show()

Both of the other tags have a pretty noticeable difficulty spike centered around 30-40 encounters (actually, for tag 23, apparently nobody's ever had more than 45 or so encounters). But is that spike a big deal?

If I use a log-log graph (graphing each axis on a logarithmic scale), it's easier to see how well a particular learning curve fits the power curve, because the power curve is supposed to be linear if we take the log of both sides:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 3))
plot_encounters(per_tag_datasets[27], label=27, ax=axes[0], limit=100, loglog=True)
plot_encounters(per_tag_datasets[23], label=23, ax=axes[1], limit=100, loglog=True)
plot_encounters(per_tag_datasets[152], label=152, ax=axes[2], limit=100, loglog=True)
[ax.legend() for ax in axes]
plt.show()

Tag 27 is still looking plausible, but the other ones... not so much. 

For tag 23, there's an aditional noticable bump between 5-10 encounters. We can see it on the linear graph, too, but it looked much less important there. To me, this patern (which - spoiler alert - does appear in a few tags) looks like users start out improving quickly over encounters 1-5 or so, and then consistently hit some kind of regression and start doing a bit worse. 

At this point, it's worth remembering that these encounters aren't exactly random or independent from each other - these users are interacting with an Intelligent Tutoring System(ITS). The system chooses quesions based on its own estimate of the user's knowledge state, and quite possibly picks harder questions for users once they've demonstrated that they're pretty good at this particular skill. This may throw a bit of a wrench in the power curve idea, but LFA is here to help - it can help us analyze which questions are harder, and separate them out into a different "skill" if necessary.

Another possibility is that the ITS actually **stops** giving questions of this type to students who are too good after 4-5 encounters. Then the data *up to* that point has these "good" students mixed into it, but the data afterwards only has students who apparently need more practice with the question. It should be pretty easy to detect if this is happening, though, and introduce some modifiers for individual student learning rates and starting points (which is, in fact, also something that LFA does).

## Plot all the tags

I wanted to get an overview of the learning curves for all the (singleton) tags. So here they are, grouped arbitrarily, limited to 100 ecounters, and smoothed with a window size of 5, on a log-log scale:

In [ ]:
def make_axes_for_many_plots(num_rows, num_cols, num_per_plot=1):
    fig, axarr = plt.subplots(num_rows, num_cols, figsize=(15, 15))
    fig.tight_layout()

    flat_axes_list = [item for sublist in axarr for item in sublist]
    # Repeat each of the num_rows*num_cols axes num_per_plot times to get a flat list to iterate through.
    axes_in_order = list(itertools.chain.from_iterable(itertools.repeat(x, num_per_plot) for x in flat_axes_list))
    return axes_in_order


In [ ]:
for t, ax in zip(singleton_tags, make_axes_for_many_plots(5, 3, num_per_plot=5)):
    plot_encounters(per_tag_datasets[t], ax=ax, limit=100, smoothing=5, label=t, loglog=True)
    ax.legend()

There are a few patterns emerging, I think:

- There are several tags with a pretty pronounced hump around encounter 30-40, as we saw in tag 152
- There are also tags with a pronounced dip around encounter 4-5, similar to tag 23 
- Many tags exhibit a rather worrying upward spike toward the end
- Some tags do follow a pretty impressive straight-line learning curve, like tag 27

I pulled out the tags that looked most like a straigth line, to see what they look like without the smoothing, and the results are still pretty impressive (especially when compared to other tags):

In [ ]:
for t in [109, 1, 147, 8, 53, 73, 173, 55, 96, 27]: # , 14, 89
    plot_encounters(per_tag_datasets[t], loglog=True, limit=100)

It's interesting that all of these have a pretty similar - and pretty shallow - slope. So the learning rate is pretty similar across these skills, but also the learning is pretty slow. After 100 encounters, you're lucky if you can reduce your odds of making a mistake by 0.3 or so.

## Logistic regression

As I mentioned above, the power curve model means that we should be able to fit a curve using logistic regression. The single input parameter will be $\log(o)$, where $o$ is the number of opportunities/encounters, and the binary event we're trying to predict is whether a mistake was made.

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression


In [ ]:
def do_error_logit(encounters_df, encounter_col = 'feedback_encounters', encounter_limit=None):
    regression_data = encounters_df[
        (encounters_df[encounter_col].notnull()) &
        (encounters_df['answered_correctly'].isin([0,1]))
    ]
    if encounter_limit is not None:
        regression_data = regression_data[regression_data[encounter_col] < encounter_limit]
                
    # Inputs for logistic regression:
    # training vector(log of total number of encounters):
    log_encs = np.log(regression_data[encounter_col]).to_numpy().reshape(-1, 1)
    # target vector(was error made?)
    error = 1-regression_data['answered_correctly']
    
    # do the regression:
    logit = LogisticRegression().fit(log_encs, error)
        
    return logit, log_encs, error
    

In [ ]:
l, X, y = do_error_logit(per_tag_datasets[27])

My `do_error_logit` helper function takes in a tag dataframe, calculates the inputs for logistic regression, does the regression using sklearn, and returns the sklearn object and the inputs it had calculated.

We can use the sklearn object to predict the probability of making an error after some number of encounters with the concept:

In [ ]:
for n_encs in [1,100,1000]:
    prediction = l.predict_proba([[np.log(n_encs)]])  # don't forget to take the log of encounters!
    print(f'Probability of making a mistake after {n_encs} encounter(s) with the concept: {prediction[0][1]:.3f}')  

There are clearly some diminishing returns there...

If you're thinkng "hang on, how is it 0.385 for 1 encounter when the graph for this tag clearly started around 0.6?" remember that sklearn outputs the **probability** of making a mistake, whereas the graphs are about the **odds**, i.e. $\frac{p}{1-p}$. If the probability is 0.385, the odds are:

In [ ]:
 0.385/(1-0.385)

Actually, sklearn gives us that $1-p$ value as part of the output of `predict_proba`. 

So with a bit of math, we can plot the output of sklearn's predictions as a learning curve,and compare it to our data:

In [ ]:
encounters = np.arange(1,500)
probabilities = l.predict_proba(np.log(encounters).reshape(-1, 1))
odds = probabilities[:,1]/probabilities[:,0] # sklearn outputs probabilities such that probabilities[:,0] == 1-probabilities[:,1] 
plt.plot(encounters, odds)
plot_encounters(per_tag_datasets[27], limit=500) 

But we don't actually need to rely on `predict_proba` every time we want to make a prediction. The sigmoid function that logistic regression fits to the data only has two parameters, which in our case are $\log(\alpha)$ and $\gamma$. We can extract those parameters from the sklearn object, and then save and re-use them whenever we need to make predictions, instead of trying to serialize and deserialize the whole sklearn logistic regression object. 

In [ ]:
def plot_prediction(sklearn_logit, original_df, limit=500, loglog=False, ax=plt, encounter_col = 'feedback_encounters'):
    log_alpha = sklearn_logit.intercept_[0]
    gamma = sklearn_logit.coef_[0,0]
    
    x = np.arange(1, limit)
    predict_y = np.exp(log_alpha + gamma*np.log(x))  # TODO: could also directly do power curve with exp(log_alpha)
    
    ax.plot(x, predict_y)
    plot_encounters(original_df, limit=limit, loglog=loglog, ax=ax, encounter_col = encounter_col) 

In [ ]:
plot_prediction(l, per_tag_datasets[27])

Yep, looks exactly the same! (of course, if I was using such a function in a separate notebook, I would make it take `log_alpha` and `gamma` as parameters, but this is more convenient for now).

What about some of the other tags?

In [ ]:
l, X, y = do_error_logit(per_tag_datasets[152])
plot_prediction(l, per_tag_datasets[152], limit=100)

In [ ]:
l, X, y = do_error_logit(per_tag_datasets[23])
plot_prediction(l, per_tag_datasets[23], limit=45)

... Not quite as good. in fact, for tag 152,the learning curve seems to be curving in the wrong direction: the skill gets *harder* as people practice more.

## Possible factors of "bad" learning curves
We're trying to aggregate data about how **users** answer **questions** into one logistic regression model. So there could be two problems with the aggregation: users and questions.

### Users

It would make sense that different users learn at different rates, and so a single learning curve doesn't fit all users very well.

I hypothesized earlier that maybe users who immediately do well on a given concept/tag aren't given any more quetions with that tag; and conversely, maybe the users who make up the long, noisy tail are those who struggle most with the concept. We can test this hypothesis by splitting users into a few different groups, based on how many total times they encounetered the question:

In [ ]:
%%time
max_per_user = per_tag_datasets[23].groupby('user_id').max()

In [ ]:
users_under_5_encounters = max_per_user[max_per_user['feedback_encounters']<5].index
users_over_30_encounters = max_per_user[max_per_user['feedback_encounters']>30].index
users_middle = max_per_user[(30>=max_per_user['feedback_encounters']) & (max_per_user['feedback_encounters']>=5)].index

In [ ]:
under_5 = per_tag_datasets[23][per_tag_datasets[23]['user_id'].isin(users_under_5_encounters)]
over_30 = per_tag_datasets[23][per_tag_datasets[23]['user_id'].isin(users_over_30_encounters)]
middle = per_tag_datasets[23][per_tag_datasets[23]['user_id'].isin(users_middle)]

In [ ]:
plt.figure(figsize=(15, 7))
plot_encounters(under_5, label='under 5')
plot_encounters(over_30, label='over 30')
plot_encounters(middle, label='between 5 and 30')
plot_encounters(per_tag_datasets[23], label='all')
plt.legend()
plt.show()

To my eyes, that does look like the split-out groups produce more reasonable curves than the all-users curve. In particular, the over-30 users definitely make for the most noisy curve, and taking them out does get rid of the upward trend toward the end. And the under-5 users are definitely doing better than the rest, right from the very first try.

Let's see if the logistic regression actually fits better:

In [ ]:
groups = {
    'under 5':under_5, 
    'over 30':over_30, 
    'middle':middle, 
    'all':per_tag_datasets[23]}
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
flat_axes_list = [item for sublist in axes for item in sublist]

for group_name, ax in zip(groups.keys(), flat_axes_list):
    group = groups[group_name]
    l, X, y = do_error_logit(group)
    plot_prediction(l, group, limit=45, ax=ax)
    ax.set_title(f'{group_name} - intercept: {np.exp(l.intercept_[0]):0.2f}; rate: {l.coef_[0,0]:0.2f}')
    

We can see that the under-5 and middle groups actually have a very similar learning rate (negative is good in this case, it means fewere mistakes). But the under-5s start out with a much lower intercept, meaning they do in fact make much fewer mistakes in the first place. The "all" curve has a much slower learning rate, so the over-30s seem to be messing thatup quite a bit.

The over-30 learning curve doesn't look great at all: it's another "backwards" curve, like they're regressing.

The fit looks decent for the under-5 curve, but still pretty noisy for the rest. This can be better seen on the log-log version of the same graphs:

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
flat_axes_list = [item for sublist in axes for item in sublist]
for group_name, ax in zip(groups.keys(), flat_axes_list):
    group = groups[group_name]
    l, X, y = do_error_logit(group)
    plot_prediction(l, group, limit=45, ax=ax, loglog=True)
    ax.set_title(f'{group_name} - intercept: {np.exp(l.intercept_[0]):0.2f}; rate: {l.coef_[0,0]:0.2f}')

Of course, it's entirely possible to do a separate logistic regression for each user. Actually, the Learning Factor Analysis paper suggests a multiple logistic regression model with some interesting simplifying assumptions (e.g. that all students learn at the same rate).

But the point of this particular exercise was to test the one specific hypothesis about whether the ITS treats different users differently. And it seems to bear out somewhat, though interestingly,it still does not account for that weird dip around 5 encounters.

A couple final notes about this particular split of the data: 
1. When I was fiddling around with this notebook, at one point I accidentally split the users on max encounters with *different* tag(152), and that actually seemed to produce some interesting results. So clustering users on some general speed/ability might be fruitful.
2. As a different strategy to separate different kinds of users, I also briefly played around with [Performance Factor Analysis](https://eric.ed.gov/?id=ED506305) - whereas LFA asks "If this student has practiced this concept N times, how likely are they to answer the next question right?", PFA asks "If this student **has gotten N questions correct and M quesions wrong so far**, how likely are they to ansswer the next question right?". It's both a bit more specific and a bit more limited (e.g. it's harder to directly incorporate lectures). But it does look very promising!
2. the under-5 users actually account for over half of the users! (see below)

In [ ]:
under_5.shape, middle.shape, over_30.shape

### Questions

If splitting up the users can't account for the weird dip-and-bump around 5-15 encounters, maybe the questions can?

If the questions being asked around encounters 5-15 are different from the ones being asked at first, that might explain the change in difficulty. If that's the case, then it might be worthwhile splitting the questions into two separate "concepts": if there's ssomething about these questions which makes them harder, it could be reasonable to treat that "something" as a separate concept to learn.


First, let's to a quick-and-dirty measure of how early or how late in the sequence of encounters each question (or lecture) is shown, on average:

In [ ]:
plt.hist(per_tag_datasets[23].groupby(['content_id']).mean()['feedback_encounters'], rwidth=0.9)
plt.show()

It does look like there's some meaningful variation in when a question is first introduced, though the range is maybe a bit narrower than I would have expected.

To get a feel for when each question typically becomes important, I want to make a stacked plot graph to answer: **at each encounter, what fraction of the total data is represented by each question?**

For better interpretability, I'll also sort the questions by the "average encounter" variable from the histogram above. Then I'd expect the earlier (easier?) questions to be at the bottom of the stacked graph, and the later ones toward the bottom.

In [ ]:
per_tag_datasets[23]['avg_encounter'] = per_tag_datasets[23].groupby(['content_id'])['feedback_encounters'].transform('mean')
easy_to_hard = per_tag_datasets[23].sort_values('avg_encounter')

In [ ]:
question_encounter_counts = easy_to_hard.groupby(['content_id', 'feedback_encounters']).count()
total_encounter_counts = easy_to_hard.groupby('feedback_encounters').count()

In [ ]:
qs = easy_to_hard['content_id'].unique()

x = total_encounter_counts.index
q_ys = np.zeros((len(qs), len(x)+1))

for num_encs, row in total_encounter_counts.iterrows():
    for i, q in enumerate(qs):
        if (q, num_encs) in question_encounter_counts.index:
            q_ys[i,int(num_encs)] = (question_encounter_counts.loc[(q, num_encs),'this_question_had_explanation'] / 
                                    total_encounter_counts.loc[num_encs]['this_question_had_explanation'])


In [ ]:
plt.figure(figsize=(15,10))
q_colors = plt.cm.get_cmap('Spectral', 13)(range(13))
plt.stackplot(np.arange(len(x)+1), q_ys, labels = [str(q) for q in qs], colors=q_colors)
plt.legend()
plt.show()


Colorful! 

I do see some clear patterns emerging:

- 3-4 of the reddish/orange questions dominate the first few encounters, with a secondary spike around 5 encounters
- The rest of the questions come in around 8 encounters and things stabilize until around 25 encounters
- between 25-40 encounters, it's kind of chaotic but the "early" reddish/orange questions come back to dominate
- after that, it's all red again forever and ever

To test the hypothesis that the first few quetions represent a different (easier) skill than the rest, we can split the question up into two groups, and re-do the whole encounters calculation for both groups (note that the encounter numbers will be quite different from the original: now encounters with the "harder" questions won't count toward the "easier" concept/skill, and vice versa)

In [ ]:
easy_qs = [4476,  4108,  6116,  6874,]
harder_qs = [ 29579, 9473,  9476,  9475,  3937,
        9474,  8394,  8395,  9472]

In [ ]:
per_tag_datasets[23]['easy_encounters'] = per_tag_datasets[23][
            (per_tag_datasets[23]['this_question_had_explanation']!=False) &
            (per_tag_datasets[23]['content_id'].isin(easy_qs))
        ].groupby('user_id').cumcount()+1

In [ ]:
per_tag_datasets[23]['hard_encounters'] = per_tag_datasets[23][
            (per_tag_datasets[23]['this_question_had_explanation']!=False) &
            (per_tag_datasets[23]['content_id'].isin(harder_qs))
        ].groupby('user_id').cumcount()+1

In [ ]:
l, X, y = do_error_logit(per_tag_datasets[23], encounter_col='easy_encounters')
plot_prediction(l, per_tag_datasets[23], limit=40,encounter_col='easy_encounters', loglog=False)
plt.title('Easier questions')
f'intercept: {np.exp(l.intercept_[0]):0.2f}; rate: {l.coef_[0,0]:0.2f}'


In [ ]:
l, X, y = do_error_logit(per_tag_datasets[23], encounter_col='hard_encounters')
plot_prediction(l, per_tag_datasets[23], limit=40, encounter_col='hard_encounters', loglog=False)
plt.title('Harder questions')
f'intercept: {np.exp(l.intercept_[0]):0.2f}; rate: {l.coef_[0,0]:0.2f}'

Just like with the user breakdown, this seems to have reduced, but not gotten rid of the dip-and-bump around 4-5 encounters. So maybe the answer is to do both.

## Side note: scores of shallow logistic curves

So far, I've avoided quantifying how good or bad the fit of the logistic regression is. And that's because **logistic regression with shallow slope can't really produce good fit scores**.

Let's take another look at the learning curves for our 3 example tags, and actually score the fit this time (against the same data we trained on):

In [ ]:
l, X, y = do_error_logit(per_tag_datasets[27])
plot_prediction(l, per_tag_datasets[27])
f'score: {l.score(X,y)}'

In [ ]:
l, X, y = do_error_logit(per_tag_datasets[152])
plot_prediction(l, per_tag_datasets[152], limit=150)
f'score: {l.score(X,y)}'

In [ ]:
l, X, y = do_error_logit(per_tag_datasets[23])
plot_prediction(l, per_tag_datasets[23], limit=50)
f'score: {l.score(X,y)}'

The difference between the scores is really not that big, compared to how much better the fit seems on tag 27 than the others (especially considering that the score is out of 1). 

But why?

Well, logistic regression predicts the probability of an event with an S-shaped sigmoid curve. When the coefficient(s) of the logisitc regression model are shallow, the slope of the S is also very shallow, and many of the predicted probabilities are closer to 0.5 than to 1 or 0. In other words, **the logistic regression predicts lots of uncertainty**.

Consider this curve of predicted probabilities from the regression for tag 23:

In [ ]:
x = np.arange(1,50)
plt.ylim(0,1)
plt.xlim(0,50)
plt.plot(x,l.predict_proba(np.log(x).reshape(-1, 1))[:,1])
plt.show()

Well... the curve *would* be S-shaped if there was such a thing as negative encounters. 

But the crucial part to consider is that as we established before, **over half of the users** had fewer than 5 total encounters with concept 23. And for these users, our model would predict somewhere betweep 0.6-0.3 chance of getting the question wrong. But that's sort of like saying "I dunno, they *might* get it right!". 

For example, consider the prediction for the first encounter of each user. The model would predict a .6 chance of error for each one of those data points. Even if this model fit the data perfectly, that would mean that at the very first encounter, 60% of the people made an error, and 40% did not. So 60% of the data points would be 1 and 40% would be 0. The accuracy score of each data point is how close the prediction was to the actual outcome. So the total accuracy for just the first encounter would be:

In [ ]:
0.6*0.6+0.4*0.4

Not great - and again, that's assuming the model fits the data perfectly! Of course, as the S-curve gets further away from the middle, the score gets better. It's actually pretty easy to simulate (approximately) the best score we could hope for: just take the input data, ignore the actual correctness values, and generate new ones based on the predicted probability from the regression. This will let us generate data that fits the regression model (nearly) perfectly, given enough data points:

In [ ]:
import random
fit_data_to_regression = per_tag_datasets[27][per_tag_datasets[27]['feedback_encounters'].notnull()]
new_X = np.log(fit_data_to_regression['feedback_encounters'].to_numpy()).reshape(-1,1)
fit_data_to_regression['prob_error']=l.predict_proba(new_X)[:,1]
fit_data_to_regression['answered_correctly'] = fit_data_to_regression['prob_error'].apply(lambda x: 1 if random.uniform(0,1) > x else 0)

In [ ]:
plot_prediction(l, fit_data_to_regression, limit=100)
f'score: {l.score(new_X,1-fit_data_to_regression["answered_correctly"])}'


The graph looks like a perfect fit, but the score is still quite far from 1.

The AUC is even worse - remember, this is the AUC for data which fits the regression curve perfectly! Essentially, this is as good as you can hope to get if the learning curve is in fact this shallow, and there is in fact this much uncertainty in how a student will answer a question.

In [ ]:
from sklearn.metrics import roc_curve, auc
def draw_roc(probabilities, labels):
    fpr, tpr, thresholds = roc_curve(labels, probabilities)
    roc_auc = auc(fpr, tpr)
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc})')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.show()
    # return thresholds


In [ ]:
draw_roc(fit_data_to_regression['prob_error'], 1-fit_data_to_regression['answered_correctly'])

# Recommendations

- Detect and separate out "noisy" users - a small fraction of users who seem to answer many, many questions but don't learn, or have an extremely noisy learning curve.
- Do some kind of logistic regresssion (like LFA or PFA) to fit learning curves to individual tags
- See if separating certain tags into "sub-tags" will improve the fit of the learning curves
- Try to reduce uncertainty by incorporating other data (if you're predicting close to 50/50 for a lot of users, there's a limit to how good you can expect the AUC to get).

# Addenda/miscellaneous

Here's some extra stuff which may or may not be interesting.

## Literally ONE GUY
Or person. Or bot.

You may have noticed that the "odds of making an error" data on tag 23 only goes up to 50 encounters or so, but the stacked plot of question frequency goes up to 80. 

Why? Well, it's only possible to calculate the odds of making an error if there was at least one correct answer at that data point (otherwise, you're dividing by 0). So it **must** mean that all encounters > 50 only have incorrect answers:


In [ ]:
per_tag_datasets[23][per_tag_datasets[23]['feedback_encounters']>50]

Yep. It's all one user. answering the same 3 questions over and over again. Incorrectly each time. In fact...

In [ ]:
per_tag_datasets[23][per_tag_datasets[23]['user_id']==1509564249].groupby('answered_correctly').count()

This user was only ever given easy qestions (easy_encounters), has answered 75 questions incorrectly, 3 correctly, and has consumed one lecture (answered_correctly=-1). This is **much** worse than random.

This must be just somebody testing the system - maybe internal test data inevitably mixing into actual production data, or maybe an actual user who's just curious what the system will do if they behave unreasonably.

Either way, this user (and other similar users) are probably really screwing up all the statistics that don't filter for behavior like this.

## Per-user LFA

Here is an interesting scatter plot of the intercepts and coefficients if we do the LFA logistic regression on a per-user basis in one of the tags. (only using users with over 100 data points, for less overfitting)

The apparent linear relationship between intercept and coefficient is curious.
The score pattern (colormap) is probably explainable with the uncertainty issue I detailed above: the model can be more certain for users who have a lower intercept, so their score tends to be higher. These are people who start out already pretty good at this concept, so the model is always pretty sure they're going to get the question right.

In [ ]:
users = per_tag_datasets[27].groupby('user_id').size()
big_users = users[users > 100]

per_user_scores = []
per_user_n = []
per_user_coef = []
per_user_intercept = []
for u in big_users.index:
    u_data = per_tag_datasets[27][per_tag_datasets[27]['user_id']==u]
    l, X, y = do_error_logit(u_data[u_data['this_question_had_explanation']==True])
    per_user_scores.append(l.score(X, y))
    per_user_n.append(big_users.loc[u])
    per_user_coef.append(l.coef_[0,0])
    per_user_intercept.append(l.intercept_[0])

In [ ]:
plt.scatter(per_user_coef, per_user_intercept, c=per_user_scores, cmap=plt.cm.coolwarm)
plt.colorbar()
plt.show()

## Why so many encounters?

Why do some tags, but not others, have so many encounters? for some tags, there are users who have answered **thousands** of questions with that tag!

Maybe they just have more data in general?..


In [ ]:
nusers = []
nencounters = []
for t in per_tag_datasets:
    nusers.append(per_tag_datasets[t]['user_id'].nunique())
    nencounters.append(per_tag_datasets[t]['feedback_encounters'].max())
plt.scatter(nusers, nencounters)
plt.show()

while the tags with **tons** of encounters do have more data points than average, there are also plenty of tags with over 150,000 data points, but under 400 encounters (in fact, most tags seem to cluster in that area). So it's not just that more people have been answering questions with these tags.

## The long, miserable tail

What kind of user would have the patience to answer thousands of questions **per tag**, and what kinds of learning results would they expect from the app?..

Here are some learning curves for users who have over 1000 encounters in tag 73. (the learning curves are smoothed with a window of 100, since otherwise there's exactly one data point per encounter, so it'd be hard to calculate odds)

In [ ]:
# poor unfortunate souls
a_tag_df = per_tag_datasets[73]
for user in a_tag_df[a_tag_df['full_encounters'] == 1000]['user_id']:
    user_encounters = a_tag_df[a_tag_df['user_id']==user]
    user_smoothed = user_encounters['answered_correctly'].rolling(100).apply(lambda a:len(a[a==0])/len(a[a==1]))
    plt.plot(range((len(user_smoothed))), user_smoothed)

These do not look like good learning curves. 

Actually, they don't look like real users to me, especially the green curve. The sharp diagonal up/down trends look like streaks of getting every single question right (upward streak) or every single question wrong (downward streak).

I think these are either also test users, or possibly accounts that are used by more than one person. For example, maybe a single account is set up on some school computers, and different students come in and practice every day.